In [ ]:
# Install required packages (runs automatically in Colab, fast no-op in Binder)
!pip install -q qiskit qiskit-aer qiskit-ibm-runtime pylatexenc matplotlib

# Hello world

{/*
  DO NOT EDIT THIS CELL!!!
  This cell's content is generated automatically by a script. Anything you add
  here will be removed next time the notebook is run. To add new content, create
  a new cell before or after this one.
*/}

<details>
<summary><b>Package versions</b></summary>

Der Code auf dieser Seite wurde mit den folgenden Anforderungen entwickelt.
Wir empfehlen, diese oder neuere Versionen zu verwenden.

```
qiskit[all]~=2.3.0
qiskit-ibm-runtime~=0.43.1
```
</details>
Dieses Beispiel besteht aus zwei Teilen. Du wirst zunächst ein einfaches Quantenprogramm erstellen und auf einer Quantenprozessoreinheit (QPU) ausführen. Da tatsächliche Quantenforschung wesentlich robustere Programme erfordert, wirst du im zweiten Abschnitt ([Skalierung auf große Qubit-Zahlen](#scale-to-large-numbers-of-qubits)) das einfache Programm auf Utility-Niveau skalieren.
## Installation und Authentifizierung
1. Falls du Qiskit noch nicht installiert hast, findest du Anweisungen im [Quickstart](/guides/quick-start) Leitfaden.

    - Installiere Qiskit Runtime, um Jobs auf Quantenhardware auszuführen:

        ```bash
        pip install qiskit-ibm-runtime
        ```

    - Richte eine Umgebung ein, um Jupyter Notebooks lokal auszuführen:

        ```bash
        pip install jupyter
        ```

2. Richte deine Authentifizierung für den Zugang zu Quantenhardware über den kostenlosen [Open Plan](/guides/plans-overview#open-plan) ein.

    (Falls du eine E-Mail-Einladung erhalten hast, einem Account beizutreten, folge stattdessen den [Schritten für eingeladene Benutzer](/guides/cloud-setup-invited).)

    - Gehe zu [IBM Quantum Platform](https://quantum.cloud.ibm.com/), um dich anzumelden oder ein Konto zu erstellen.
         > **Note:** Wenn du dich über einen Proxy-Server verbindest, musst du Qiskit Runtime v0.44.0 oder neuer verwenden.
    - Generiere deinen API-Schlüssel (auch als *API-Token* bezeichnet) im [Dashboard](https://quantum.cloud.ibm.com/) und kopiere ihn an einen sicheren Ort.
    - Gehe zur [Instances](https://quantum.cloud.ibm.com/instances) Seite und finde die Instanz, die du verwenden möchtest. Bewege den Mauszeiger über deren CRN und klicke, um sie zu kopieren.

    - Speichere deine Zugangsdaten lokal mit diesem Code:

        ```python
        from qiskit_ibm_runtime import QiskitRuntimeService

        QiskitRuntimeService.save_account(
        token="<your-api-key>", # Use the 44-character API_KEY you created and saved from the IBM Quantum Platform Home dashboard
        instance="<CRN>", # Optional
        )
        ```

3. Nun kannst du jederzeit diesen Python-Code verwenden, wenn du dich beim Qiskit Runtime Service authentifizieren möchtest:
    ```python
        from qiskit_ibm_runtime import QiskitRuntimeService

        # Run every time you need the service
        service = QiskitRuntimeService()
    ```
> **Info:** Falls du einen öffentlichen Computer oder eine andere unsichere Umgebung verwendest, folge stattdessen den [Anweisungen zur manuellen Authentifizierung](/guides/cloud-setup-untrusted), um deine Zugangsdaten zu schützen.
## Erstellen und Ausführen eines einfachen Quantenprogramms
Die vier Schritte zum Schreiben eines Quantenprogramms mit Qiskit Patterns sind:

1.  Problem in ein quantennatives Format abbilden.

2.  Schaltkreise und Operatoren optimieren.

3.  Ausführung mit einer Quantum Primitive Funktion.

4.  Ergebnisse analysieren.

### Schritt 1. Problem in ein quantennatives Format abbilden
In einem Quantenprogramm sind *Quantenschaltkreise* das native Format zur Darstellung von Quanteninstruktionen, und *Operatoren* repräsentieren die zu messenden Observablen. Beim Erstellen eines Schaltkreises erstellst du normalerweise ein neues [`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#quantumcircuit-class) Objekt und fügst ihm dann sequenziell Instruktionen hinzu.
Die folgende Code-Zelle erstellt einen Schaltkreis, der einen *Bell-Zustand* erzeugt, ein Zustand, in dem zwei Qubits vollständig miteinander verschränkt sind.

> **Note:** Das Qiskit SDK verwendet die LSb 0 Bit-Nummerierung, wobei das $n^{th}$ Digit den Wert $1 \ll n$ oder $2^n$ hat. Weitere Details findest du im Thema [Bit-Ordering im Qiskit SDK](/guides/bit-ordering).

In [ ]:
from qiskit import QuantumCircuit
from qiskit.quantum_info import SparsePauliOp
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService
from qiskit_ibm_runtime import EstimatorOptions
from qiskit_ibm_runtime import EstimatorV2 as Estimator
from matplotlib import pyplot as plt
# Uncomment the next line if you want to use a simulator:
# from qiskit_ibm_runtime.fake_provider import FakeBelemV2


# Create a new circuit with two qubits
qc = QuantumCircuit(2)

# Add a Hadamard gate to qubit 0
qc.h(0)

# Perform a controlled-X gate on qubit 1, controlled by qubit 0
qc.cx(0, 1)

# Return a drawing of the circuit using MatPlotLib ("mpl").
# These guides are written by using Jupyter notebooks, which
# display the output of the last line of each cell.
# If you're running this in a script, use `print(qc.draw())` to
# print a text drawing.
qc.draw("mpl")

<Image src="../docs/images/guides/hello-world/extracted-outputs/930ca3b6-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/hello-world/extracted-outputs/930ca3b6-0.svg)

Die Dokumentation für [`QuantumCircuit`](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit#quantumcircuit-class) enthält alle verfügbaren Operationen.
Beim Erstellen von Quantenschaltkreisen musst du auch berücksichtigen, welche Art von Daten du nach der Ausführung zurückerhalten möchtest. Qiskit bietet zwei Möglichkeiten, Daten zurückzugeben: Du kannst eine Wahrscheinlichkeitsverteilung für eine Menge von Qubits erhalten, die du zu messen wählst, oder du kannst den Erwartungswert einer Observablen erhalten. Bereite deine Workload vor, um deinen Schaltkreis auf eine dieser beiden Arten mit [Qiskit Primitives](/guides/get-started-with-primitives) zu messen (detailliert erklärt in [Schritt 3](#step-3-execute-using-the-quantum-primitives)).

Dieses Beispiel misst Erwartungswerte unter Verwendung des `qiskit.quantum_info` Submoduls, das durch Operatoren spezifiziert wird (mathematische Objekte zur Darstellung einer Aktion oder eines Prozesses, der einen Quantenzustand verändert). Die folgende Code-Zelle erstellt sechs Zwei-Qubit-Pauli-Operatoren: `IZ`, `IX`, `ZI`, `XI`, `ZZ` und `XX`.

In [2]:
# Set up six different observables.

observables_labels = ["IZ", "IX", "ZI", "XI", "ZZ", "XX"]
observables = [SparsePauliOp(label) for label in observables_labels]

> **Note:** Hier ist etwas wie der `ZZ` Operator eine Kurzschreibweise für das Tensorprodukt $Z\otimes Z$, was bedeutet, Z auf Qubit 1 und Z auf Qubit 0 zusammen zu messen und Informationen über die Korrelation zwischen Qubit 1 und Qubit 0 zu erhalten. Erwartungswerte wie diese werden typischerweise auch als $\langle Z_1 Z_0 \rangle$ geschrieben.
> 
> Wenn der Zustand verschränkt ist, dann sollte die Messung von $\langle Z_1 Z_0 \rangle$ anders sein als die Messung von $\langle I_1 \otimes Z_0 \rangle \langle Z_1 \otimes I_0 \rangle$. Für den spezifischen verschränkten Zustand, der durch unseren oben beschriebenen Schaltkreis erstellt wird, sollte die Messung von $\langle Z_1 Z_0 \rangle$ 1 sein und die Messung von $\langle I_1 \otimes Z_0 \rangle \langle Z_1 \otimes I_0 \rangle$ sollte null sein.
<span id="optimize"></span>

### Schritt 2. Schaltkreise und Operatoren optimieren

Beim Ausführen von Schaltkreisen auf einem Gerät ist es wichtig, die Menge der Instruktionen, die der Schaltkreis enthält, zu optimieren und die Gesamttiefe (ungefähr die Anzahl der Instruktionen) des Schaltkreises zu minimieren. Dies stellt sicher, dass du die bestmöglichen Ergebnisse erzielst, indem du die Auswirkungen von Fehlern und Rauschen reduzierst. Zusätzlich müssen die Instruktionen des Schaltkreises der [Instruction Set Architecture (ISA)](/guides/transpile#instruction-set-architecture) eines Backend-Geräts entsprechen und dessen Basis-Gates und Qubit-Konnektivität berücksichtigen.

Der folgende Code instanziiert ein echtes Gerät, an das ein Job übermittelt werden soll, und transformiert den Schaltkreis und die Observablen, um mit der ISA dieses Backends übereinzustimmen. Dies setzt voraus, dass du bereits [deine Zugangsdaten gespeichert hast](/guides/cloud-setup)

In [ ]:
service = QiskitRuntimeService()

backend = service.least_busy(simulator=False, operational=True)

# Convert to an ISA circuit and layout-mapped observables.
pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
isa_circuit = pm.run(qc)

isa_circuit.draw("mpl", idle_wires=False)

<Image src="../docs/images/guides/hello-world/extracted-outputs/9a901271-0.svg" alt="Output of the previous code cell" />

![Output of the previous code cell](../docs/images/guides/hello-world/extracted-outputs/9a901271-0.svg)

### Schritt 3. Ausführung mit den Quantum Primitives
Quantencomputer können zufällige Ergebnisse produzieren, daher sammelst du normalerweise eine Stichprobe der Ausgaben, indem du den Schaltkreis viele Male ausführst. Du kannst den Wert der Observablen mit der `Estimator` Klasse schätzen. `Estimator` ist eines von zwei [Primitives](/guides/get-started-with-primitives); das andere ist `Sampler`, der verwendet werden kann, um Daten von einem Quantencomputer zu erhalten. Diese Objekte besitzen eine `run()` Methode, die die Auswahl von Schaltkreisen, Observablen und Parametern (falls zutreffend) unter Verwendung eines [Primitive Unified Bloc (PUB)](/guides/primitives#sampler) ausführt.

In [4]:
# Construct the Estimator instance.

estimator = Estimator(mode=backend)
estimator.options.resilience_level = 1
estimator.options.default_shots = 5000

mapped_observables = [
    observable.apply_layout(isa_circuit.layout) for observable in observables
]

# One pub, with one circuit to run against five different observables.
job = estimator.run([(isa_circuit, mapped_observables)])

# Use the job ID to retrieve your job data later
print(f">>> Job ID: {job.job_id()}")

>>> Job ID: d5k96q4jt3vs73ds5tgg


After a job is submitted, you can wait until either the job is completed within your current python instance, or use the `job_id` to retrieve the data at a later time.  (See the [section on retrieving jobs](/docs/guides/monitor-job#retrieve-job-results-at-a-later-time) for details.)

After the job completes, examine its output through the job's `result()` attribute.

In [5]:
# This is the result of the entire submission.  You submitted one Pub,
# so this contains one inner result (and some metadata of its own).
job_result = job.result()

# This is the result from our single pub, which had six observables,
# so contains information on all six.
pub_result = job.result()[0]

In [6]:
# Check there are six observables.
# If not, edit the comments in the previous cell and update this test.
assert len(pub_result.data.evs) == 6

<Admonition type="note" title="Alternative: run the example using a simulator">
  When you run your quantum program on a real device, your workload must wait in a queue before it runs. To save time, you can instead use the following code to run this small workload on the [`fake_provider`](../api/qiskit-ibm-runtime/fake-provider) with the Qiskit Runtime local testing mode. Note that this is only possible for a small circuit. When you scale up in the next section, you will need to use a real device.

  ```python

  # Use the following code instead if you want to run on a simulator:

  from qiskit_ibm_runtime.fake_provider import FakeBelemV2
  backend = FakeBelemV2()
  estimator = Estimator(backend)

  # Convert to an ISA circuit and layout-mapped observables.

  pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
  isa_circuit = pm.run(qc)
  mapped_observables = [
      observable.apply_layout(isa_circuit.layout) for observable in observables
  ]

  job = estimator.run([(isa_circuit, mapped_observables)])
  result = job.result()

  # This is the result of the entire submission.  You submitted one Pub,
  # so this contains one inner result (and some metadata of its own).

  job_result = job.result()

  # This is the result from our single pub, which had five observables,
  # so contains information on all five.

  pub_result = job.result()[0]
  ```
</Admonition>

### Step 4. Analyze the results

The analyze step is typically where you might post-process your results using, for example, measurement error mitigation or zero noise extrapolation (ZNE). You might feed these results into another workflow for further analysis or prepare a plot of the key values and data. In general, this step is specific to your problem.  For this example, plot each of the expectation values that were measured for our circuit.

The expectation values and standard deviations for the observables you specified to Estimator are accessed through the job result's `PubResult.data.evs` and `PubResult.data.stds` attributes. To obtain the results from Sampler, use the `PubResult.data.meas.get_counts()` function, which will return a `dict` of measurements in the form of bitstrings as keys and counts as their corresponding values. For more information, see [Get started with Sampler.](/docs/guides/get-started-with-primitives#get-started-with-sampler)

In [ ]:
# Plot the result

values = pub_result.data.evs

errors = pub_result.data.stds

# plotting graph
plt.plot(observables_labels, values, "-o")
plt.xlabel("Observables")
plt.ylabel("Values")
plt.show()

<Image src="../docs/images/guides/hello-world/extracted-outputs/87143fcc-0.svg" alt="Output of the previous code cell" />

> **Note:** Wenn du dein Quantenprogramm auf einem echten Gerät ausführst, muss deine Workload in einer Warteschlange warten, bevor sie ausgeführt wird. Um Zeit zu sparen, kannst du stattdessen den folgenden Code verwenden, um diese kleine Workload auf dem [`fake_provider`](../api/qiskit-ibm-runtime/fake-provider) mit dem Qiskit Runtime Local Testing Mode auszuführen. Beachte, dass dies nur für einen kleinen Schaltkreis möglich ist. Wenn du im nächsten Abschnitt skalierst, musst du ein echtes Gerät verwenden.
> 
>   ```python
> 
>   # Use the following code instead if you want to run on a simulator:
> 
>   from qiskit_ibm_runtime.fake_provider import FakeBelemV2
>   backend = FakeBelemV2()
>   estimator = Estimator(backend)
> 
>   # Convert to an ISA circuit and layout-mapped observables.
> 
>   pm = generate_preset_pass_manager(backend=backend, optimization_level=1)
>   isa_circuit = pm.run(qc)
>   mapped_observables = [
>       observable.apply_layout(isa_circuit.layout) for observable in observables
>   ]
> 
>   job = estimator.run([(isa_circuit, mapped_observables)])
>   result = job.result()
> 
>   # This is the result of the entire submission.  You submitted one Pub,
>   # so this contains one inner result (and some metadata of its own).
> 
>   job_result = job.result()
> 
>   # This is the result from our single pub, which had five observables,
>   # so contains information on all five.
> 
>   pub_result = job.result()[0]
>   ```
### Schritt 4. Ergebnisse analysieren
Der Analyseschritt ist typischerweise der Ort, an dem du deine Ergebnisse nachbearbeiten könntest, beispielsweise unter Verwendung von Measurement Error Mitigation oder Zero Noise Extrapolation (ZNE). Du könntest diese Ergebnisse in einen anderen Workflow zur weiteren Analyse einspeisen oder ein Diagramm der Schlüsselwerte und Daten erstellen. Im Allgemeinen ist dieser Schritt spezifisch für dein Problem. Für dieses Beispiel zeichnest du jeden der Erwartungswerte, die für unseren Schaltkreis gemessen wurden.

Die Erwartungswerte und Standardabweichungen für die Observablen, die du für Estimator spezifiziert hast, werden über die `PubResult.data.evs` und `PubResult.data.stds` Attribute des Job-Ergebnisses aufgerufen. Um die Ergebnisse von Sampler zu erhalten, verwende die `PubResult.data.meas.get_counts()` Funktion, die ein `dict` von Messungen in Form von Bitstrings als Schlüssel und Zählungen als deren entsprechende Werte zurückgibt. Weitere Informationen findest du unter [Erste Schritte mit Sampler](/guides/get-started-with-primitives#get-started-with-sampler).

In [8]:
# Make sure the results follow the claim from the previous markdown cell.
# This can happen when the device occasionally behaves strangely. If this cell
# fails, you may just need to run the notebook again.

_results = {obs: val for obs, val in zip(observables_labels, values)}
for _label in ["IZ", "IX", "ZI", "XI"]:
    assert abs(_results[_label]) < 0.2
for _label in ["XX", "ZZ"]:
    assert _results[_label] > 0.8

![Output of the previous code cell](../docs/images/guides/hello-world/extracted-outputs/87143fcc-0.svg)

Beachte, dass für die Qubits 0 und 1 die unabhängigen Erwartungswerte sowohl von X als auch von Z 0 sind, während die Korrelationen (`XX` und `ZZ`) 1 sind. Dies ist ein Kennzeichen von Quantenverschränkung.

In [ ]:
def get_qc_for_n_qubit_GHZ_state(n: int) -> QuantumCircuit:
    """This function will create a qiskit.QuantumCircuit (qc) for an n-qubit GHZ state.

    Args:
        n (int): Number of qubits in the n-qubit GHZ state

    Returns:
        QuantumCircuit: Quantum circuit that generate the n-qubit GHZ state, assuming all qubits start in the 0 state
    """
    if isinstance(n, int) and n >= 2:
        qc = QuantumCircuit(n)
        qc.h(0)
        for i in range(n - 1):
            qc.cx(i, i + 1)
    else:
        raise Exception("n is not a valid input")
    return qc


# Create a new circuit with two qubits (first argument) and two classical
# bits (second argument)
n = 100
qc = get_qc_for_n_qubit_GHZ_state(n)

## Skalierung auf große Qubit-Zahlen
Im Quantencomputing ist Arbeit auf Utility-Niveau entscheidend für den Fortschritt im Feld. Solche Arbeit erfordert Berechnungen in einem viel größeren Maßstab; die Arbeit mit Schaltkreisen, die über 100 Qubits und über 1000 Gates verwenden können. Dieses Beispiel demonstriert, wie du Arbeit auf Utility-Niveau auf IBM&reg; QPUs durchführen kannst, indem du einen 100-Qubit-GHZ-Zustand erstellst und analysierst. Es verwendet den Qiskit Patterns Workflow und endet mit der Messung des Erwartungswerts $\langle Z_0 Z_i \rangle $ für jedes Qubit.

### Schritt 1. Problem abbilden
Schreibe eine Funktion, die einen `QuantumCircuit` zurückgibt, der einen $n$-Qubit-GHZ-Zustand vorbereitet (im Wesentlichen ein erweiterter Bell-Zustand), und verwende dann diese Funktion, um einen 100-Qubit-GHZ-Zustand vorzubereiten und die zu messenden Observablen zu sammeln.

In [ ]:
# ZZII...II, ZIZI...II, ... , ZIII...IZ
operator_strings = [
    "Z" + "I" * i + "Z" + "I" * (n - 2 - i) for i in range(n - 1)
]
print(operator_strings)
print(len(operator_strings))

operators = [SparsePauliOp(operator) for operator in operator_strings]

['ZZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII', 'ZIIIIIIIIIZIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIIII

Als Nächstes bildest du auf die interessierenden Operatoren ab. Dieses Beispiel verwendet die `ZZ` Operatoren zwischen Qubits, um das Verhalten zu untersuchen, während sie sich weiter voneinander entfernen. Zunehmend ungenaue (verfälschte) Erwartungswerte zwischen entfernten Qubits würden den vorhandenen Rauschpegel offenbaren.

In [ ]:
service = QiskitRuntimeService()

backend = service.least_busy(
    simulator=False, operational=True, min_num_qubits=100
)
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)

isa_circuit = pm.run(qc)
isa_operators_list = [op.apply_layout(isa_circuit.layout) for op in operators]

### Step 3. Execute on hardware

Submit the job and enable error suppression by using a technique to reduce errors called [dynamical decoupling.](../api/qiskit-ibm-runtime/options-dynamical-decoupling-options) The resilience level specifies how much resilience to build against errors. Higher levels generate more accurate results, at the expense of longer processing times.  For further explanation of the options set in the following code, see [Configure error mitigation for Qiskit Runtime.](/docs/guides/configure-error-mitigation)

In [ ]:
options = EstimatorOptions()
options.resilience_level = 1
options.dynamical_decoupling.enable = True
options.dynamical_decoupling.sequence_type = "XY4"

# Create an Estimator object
estimator = Estimator(backend, options=options)

In [13]:
# Submit the circuit to Estimator
job = estimator.run([(isa_circuit, isa_operators_list)])
job_id = job.job_id()
print(job_id)

d5k9mmqvcahs73a1ni3g


### Schritt 3. Auf Hardware ausführen
Übermittle den Job und aktiviere die Fehlerunterdrückung, indem du eine Technik zur Reduzierung von Fehlern namens [Dynamical Decoupling](../api/qiskit-ibm-runtime/options-dynamical-decoupling-options) verwendest. Das Resilience-Level gibt an, wie viel Resilienz gegen Fehler aufgebaut werden soll. Höhere Level erzeugen genauere Ergebnisse, auf Kosten längerer Verarbeitungszeiten. Für weitere Erklärungen der Optionen, die im folgenden Code gesetzt werden, siehe [Fehlermitigation für Qiskit Runtime konfigurieren](/guides/configure-error-mitigation).

In [ ]:
# data
data = list(range(1, len(operators) + 1))  # Distance between the Z operators
result = job.result()[0]
values = result.data.evs  # Expectation value at each Z operator.
values = [
    v / values[0] for v in values
]  # Normalize the expectation values to evaluate how they decay with distance.

# plotting graph
plt.plot(data, values, marker="o", label="100-qubit GHZ state")
plt.xlabel("Distance between qubits $i$")
plt.ylabel(r"$\langle Z_i Z_0 \rangle / \langle Z_1 Z_0 \rangle $")
plt.legend()
plt.show()

<Image src="../docs/images/guides/hello-world/extracted-outputs/de91ebd0-0.svg" alt="Output of the previous code cell" />

The previous plot shows that as the distance between qubits increases, the signal decays because of the presence of noise.

## Next steps

<Admonition type="tip" title="Recommendations">
  -   Try one of these tutorials:
      - [Ground-state energy estimation of the Heisenberg chain with VQE](/docs/tutorials/spin-chain-vqe)
      - Solve optimization problems using [QAOA](/docs/tutorials/quantum-approximate-optimization-algorithm)
      - Train [quantum kernel](/docs/tutorials/quantum-kernel-training) models for machine learning tasks
  - Find detailed installation instructions in the [Install Qiskit](/docs/guides/install-qiskit) guide.
  - If you prefer not to install Qiskit locally, read about options to use Qiskit in an [online development environment.](/docs/guides/online-lab-environments)
  - To save multiple account credentials or to specify other account options, see detailed instructions in the [Save your login credentials](/docs/guides/save-credentials#save-your-access-credentials) guide.
</Admonition>